##  Importing all the libraries

In [1]:
import prometheus
import os
%load_ext dotenv
%dotenv

## Setting up the connection to scrape metrics from the Thanos instance
We need to obtain a session token in order to scrape the required Thanos instance on OpenShift. You can obtain this token value (after having logged in to OpenShift) by executing the following command in your local terminal: $oc whoami -t and setting it as an env variable.

## Collecting the metrics 


In [2]:
url = os.environ.get("THANOS_PROM_URL")
token = os.environ.get("PROM_SESSION_TOKEN")

In [3]:
prom = prometheus.Prometheus(url=url, token=token)

In [4]:
all_metrics = prom.all_metrics()

In [5]:
all_metrics

['alerts',
 'cluster_operator_conditions',
 'cluster_operator_up',
 'cluster_version',
 'cluster_version_available_updates',
 'cluster_version_payload',
 'etcd_object_counts',
 'machine_cpu_cores',
 'machine_memory_bytes',
 'scrape_duration_seconds',
 'scrape_samples_post_metric_relabeling',
 'scrape_samples_scraped',
 'up']

In [6]:
print("Number of metrics present:", len(all_metrics))

Number of metrics present: 13


We can fetch the data for a specific metric name based on a particular time frame, duration size and label configurations

In [30]:
metric_data = prom.get_metric_range_data(metric_name='etcd_object_counts', start_time='10m', chunk_size='5m')

In [31]:
len(metric_data)

160433

In [32]:
#Exploring one among all the metrics collected
metric_data[0].keys()

dict_keys(['metric', 'values'])

In [33]:
metadata = metric_data[0].get("metric")

In [34]:
metadata

{'__name__': 'etcd_object_counts',
 '_id': '00725312-cfc1-4acc-a968-9a15909fefa0',
 'endpoint': 'https',
 'instance': '10.0.134.228:6443',
 'job': 'apiserver',
 'monitor': 'prometheus',
 'namespace': 'default',
 'pod': 'telemeter-server-3',
 'prometheus': 'openshift-monitoring/k8s',
 'prometheus_replica': 'prometheus-k8s-1',
 'replica': '$(HOSTNAME)',
 'resource': 'apiservers.config.openshift.io',
 'service': 'kubernetes'}

In [35]:
#Obtaining the labels of the metric
labels = list(metadata.keys())
print("Number of labels identified:", len(labels))

Number of labels identified: 13


In [36]:
labels

['__name__',
 '_id',
 'endpoint',
 'instance',
 'job',
 'monitor',
 'namespace',
 'pod',
 'prometheus',
 'prometheus_replica',
 'replica',
 'resource',
 'service']

In [37]:
#Obtaining the metric values
metric_values = metric_data[0].get('values')
metric_values

[[1552676388.975, '0']]